**Summary of the Experiment:**

Dataset Name : CommonsenseQA

There were two files given namely train_rand_split.jsonl and dev_rand_split.jsonl

train_rand_split is used as training set and dev_rand_split as Validation set

Data contains following columns:
1. Question id
2. Question
3. Answer key

Following data pre-processing steps are carried out:

1. Question column contains the context followed by question itself was captured in the form of dictionary : Here context and question has broken into two separate columns as question includes context as well

2. The Answer key contains either of one of the options from [A,B,C,D,E] as it's just a label captured in the form of character and do not reveal much infomation about the answer itself. So capture the actual answer text label key is mapped to appropriate answer text 

Source : Question
Target : Answer text

Changes made to the code : Added glove emmbedding while building a vocab

Final observations : Model is overfitting and need addtional architectural changes to imporove the performance


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/END-NLP/Assignment9-encoder-decoder

/content/drive/My Drive/END-NLP/Assignment9-encoder-decoder


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd 
from torchtext.data import Example,Field, BucketIterator,Dataset

import spacy
import numpy as np

import random
import math
import time

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


## Reading CommonsenseQA data set (saved in the form of json)

In [ ]:
train=pd.read_json('train_rand_split.jsonl',lines=True)
test=pd.read_json('dev_rand_split.jsonl',lines=True)

In [ ]:
def fun_map_answerkey(dt):
    l=list(dt['question'].values())[1]
    for i in l:
      if(dt['answerKey'] in i.values()):
        return(str(list(i.values())[1]))

In [ ]:
train['ans_text']=train[['question','answerKey']].apply(fun_map_answerkey,axis=1)

In [ ]:
def fun_unlist(a):
    l=list(a.values())
    l1=[ x['label'] +"." + x['text'] for x in l[1:][0]]
    return(pd.Series([ str(l[2]+" "+" ".join(l1)), str(l[0])]))


In [ ]:
train[['question_extracted','question_concept']]=train['question'].apply(lambda x : fun_unlist(x))

In [ ]:
train.head()

,answerKey,id,question,ans_text,question_extracted,question_concept
0,A,075e483d21c29a511267ef62bedc0461,"{'question_concept': 'punishing', 'choices': [{'label': 'A', 'text': 'ignore'}, {'label': 'B', 'text': 'enforce'}, {'label': 'C', 'text': 'authoritarian'}, {'label': 'D', 'text': 'yell at'}, {'label': 'E', 'text': 'avoid'}], 'stem': 'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?'}",ignore,"The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change? A.ignore B.enforce C.authoritarian D.yell at E.avoid",punishing
1,B,61fe6e879ff18686d7552425a36344c8,"{'question_concept': 'people', 'choices': [{'label': 'A', 'text': 'race track'}, {'label': 'B', 'text': 'populated areas'}, {'label': 'C', 'text': 'the desert'}, {'label': 'D', 'text': 'apartment'}, {'label': 'E', 'text': 'roadblock'}], 'stem': 'Sammy wanted to go to where the people were. Where might he go?'}",populated areas,Sammy wanted to go to where the people were. Where might he go? A.race track B.populated areas C.the desert D.apartment E.roadblock,people
2,A,4c1cb0e95b99f72d55c068ba0255c54d,"{'question_concept': 'choker', 'choices': [{'label': 'A', 'text': 'jewelry store'}, {'label': 'B', 'text': 'neck'}, {'label': 'C', 'text': 'jewlery box'}, {'label': 'D', 'text': 'jewelry box'}, {'label': 'E', 'text': 'boutique'}], 'stem': 'To locate a choker not located in a jewelry box or boutique where would you go?'}",jewelry store,To locate a choker not located in a jewelry box or boutique where would you go? A.jewelry store B.neck C.jewlery box D.jewelry box E.boutique,choker
3,D,02e821a3e53cb320790950aab4489e85,"{'question_concept': 'highway', 'choices': [{'label': 'A', 'text': 'united states'}, {'label': 'B', 'text': 'mexico'}, {'label': 'C', 'text': 'countryside'}, {'label': 'D', 'text': 'atlas'}, {'label': 'E', 'text': 'oceans'}], 'stem': 'Google Maps and other highway and street GPS services have replaced what?'}",atlas,Google Maps and other highway and street GPS services have replaced what? A.united states B.mexico C.countryside D.atlas E.oceans,highway
4,C,23505889b94e880c3e89cff4ba119860,"{'question_concept': 'fox', 'choices': [{'label': 'A', 'text': 'pretty flowers.'}, {'label': 'B', 'text': 'hen house'}, {'label': 'C', 'text': 'natural habitat'}, {'label': 'D', 'text': 'storybook'}, {'label': 'E', 'text': 'dense forest'}], 'stem': 'The fox walked from the city into the forest, what was it looking for?'}",natural habitat,"The fox walked from the city into the forest, what was it looking for? A.pretty flowers. B.hen house C.natural habitat D.storybook E.dense forest",fox


In [ ]:
test['ans_text']=test[['question','answerKey']].apply(fun_map_answerkey,axis=1)
test[['question_extracted','question_concept']]=test['question'].apply(lambda x : fun_unlist(x))

## Obtaining only revelent fields from the data set

In [ ]:
train_cols=train[['question_extracted','ans_text']]
test_cols=test[['question_extracted','ans_text']]

In [ ]:
train_cols.shape

(9741, 2)

In [ ]:
test_cols.shape

(1221, 2)

### Create our fields to process our data

In [ ]:
question = Field(tokenize='spacy', 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)
answer = Field(tokenize='spacy', 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

In [ ]:
fields = [('questions', question),('answers',answer)]

In [ ]:
fields

[('questions', <torchtext.data.field.Field at 0x7fecb5026f28>),
 ('answers', <torchtext.data.field.Field at 0x7fecb4e82a90>)]

In [ ]:
example = [Example.fromlist([train_cols.question_extracted[i],train_cols.ans_text[i]], fields) for i in range(train_cols.shape[0])]

In [ ]:
questionDataset = Dataset(example, fields)

In [ ]:
#(train, valid) = questionDataset.split(split_ratio=[0.80, 0.20], random_state=random.seed(SEED))

In [ ]:
train=questionDataset
print(len(train))

9741


In [ ]:
print(vars(train.examples[0]))

{'questions': ['the', 'sanctions', 'against', 'the', 'school', 'were', 'a', 'punishing', 'blow', ',', 'and', 'they', 'seemed', 'to', 'what', 'the', 'efforts', 'the', 'school', 'had', 'made', 'to', 'change', '?', 'a.ignore', 'b.enforce', 'c.authoritarian', 'd.yell', 'at', 'e.avoid'], 'answers': ['ignore']}


In [ ]:
example1 = [Example.fromlist([test_cols.question_extracted[i],test_cols.ans_text[i]], fields) for i in range(test_cols.shape[0])]
valid = Dataset(example1, fields)

In [ ]:
len(valid)

1221

In [ ]:
MAX_VOCAB_SIZE = 25_000

question.build_vocab(train,max_size = MAX_VOCAB_SIZE,
                     vectors = "glove.6B.100d",unk_init = torch.Tensor.normal_)
answer.build_vocab(train,max_size = MAX_VOCAB_SIZE,
                     vectors = "glove.6B.100d",unk_init = torch.Tensor.normal_)


.vector_cache/glove.6B.zip: 862MB [06:29, 2.21MB/s]                           
100%|█████████▉| 398745/400000 [00:20<00:00, 20717.65it/s]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator = BucketIterator.splits(
    (train, valid), 
    batch_size = BATCH_SIZE, 
    device = device,sort=False)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]

        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
INPUT_DIM = len(question.vocab)
OUTPUT_DIM = len(answer.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
OUTPUT_DIM

3823

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(24877, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(3823, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=3823, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = answer.vocab.stoi[answer.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.questions
        trg = batch.answers
              
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.questions
            trg = batch.answers

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 3s
	Train Loss: 5.351 | Train PPL: 210.722
	 Val. Loss: 4.970 |  Val. PPL: 143.996
Epoch: 02 | Time: 0m 2s
	Train Loss: 4.715 | Train PPL: 111.602
	 Val. Loss: 4.958 |  Val. PPL: 142.282
Epoch: 03 | Time: 0m 2s
	Train Loss: 4.602 | Train PPL:  99.653
	 Val. Loss: 5.017 |  Val. PPL: 150.884
Epoch: 04 | Time: 0m 2s
	Train Loss: 4.562 | Train PPL:  95.739
	 Val. Loss: 5.012 |  Val. PPL: 150.144
Epoch: 05 | Time: 0m 2s
	Train Loss: 4.517 | Train PPL:  91.520
	 Val. Loss: 5.049 |  Val. PPL: 155.877
Epoch: 06 | Time: 0m 2s
	Train Loss: 4.496 | Train PPL:  89.674
	 Val. Loss: 5.075 |  Val. PPL: 159.910
Epoch: 07 | Time: 0m 2s
	Train Loss: 4.473 | Train PPL:  87.613
	 Val. Loss: 5.087 |  Val. PPL: 161.855
Epoch: 08 | Time: 0m 2s
	Train Loss: 4.428 | Train PPL:  83.741
	 Val. Loss: 5.103 |  Val. PPL: 164.480
Epoch: 09 | Time: 0m 2s
	Train Loss: 4.397 | Train PPL:  81.169
	 Val. Loss: 5.139 |  Val. PPL: 170.515
Epoch: 10 | Time: 0m 2s
	Train Loss: 4.377 | Train PPL:  79.584


In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'\t Test  Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}% \n')

	 Test  Loss: 0.000 |  Test Acc: 100.00% 

